<a href="https://colab.research.google.com/github/datajcthemax/playdata/blob/main/5_31_airflow_variable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
month= datetime.now().strftime("%m")
day= datetime.now().strftime("%d")

ETL_DAG = Variable.get("ETL_DAG")
LOG_PATH = Variable.get("TO_SERVER_LOG")
SAVE_LOG_PATH = Variable.get("SAVE_LOG_PATH")
LOCAL_PATH = Variable.get("LOCAL_PATH")
UPLOAD_PATH = Variable.get("TO_SERVER_PATH")

default_args = {
    'owner': 'airflow',


# Define the BashOperator task
start = BashOperator(
    task_id='server_access',
    bash_command=f"""
        if pip list | grep "awscli"
        then
        echo "pass this process"
        else
        sh {ETL_DAG}/key_config.sh
        echo "process confirmed"
        fi
        """,


send_SUM = BashOperator(
    task_id='send_SUM',
    bash_command=f'aws s3 cp {LOCAL_PATH}/SUM.LOG {UPLOAD_PATH}/{today}/SUM.LOG',
    dag=test_dag
)
send_RAW = BashOperator(
    task_id='send_RAW',
    bash_command=f'aws s3 cp {LOCAL_PATH}/RAW.LOG {UPLOAD_PATH}/{day}/{month}/{year}/RAW.LOG',
    dag=test_dag
)
check_DONE = BashOperator(
    task_id='check_DONE',
    bash_command=f"{ETL_DAG}/check.sh {UPLOAD_PATH}/DONE/{today} _DONE",
    retries=5,
    retry_delay=timedelta(minutes=1),
    dag=test_dag
)
gen_DONE = BashOperator(
    task_id='generate_DONE',
    bash_command=f'aws s3 cp {LOCAL_PATH}/_DONE {UPLOAD_PATH}/DONE/{today}/_DONE',
    dag=test_dag
)


start >> cp_log 
cp_log >> gen_RAW 
gen_RAW>> send_RAW 
gen_RAW>>gen_SUM >> send_SUM >> gen_DONE >> check_DONE >> end